## 위치로만 인자를 지정하게 하거나 키워드로만 인자를 지정하게 해서 함수 호출을 명확하게 만들라

키워드를 사용해 인자를 넘기는 기능은 파이썬 함수의 강력한 기능이다.

인자의 유연성을 활용하면 코드를 처음 읽는 사람도 더 명확하게 용례를 이해할 수 있는 함수를 작성할 수 있다.

예를 들어 한 숫자를 다른 숫자로 나눌 때 세삼하게 주의를 기울여야 한느 특별한 경우가 있는지 알고 싶다고 하자.

떄로는 ZeroDivisionError 예외를 무시하고 무한대를 반환하고 싶고, 어떨 때는 OverflowError 예외를 무시하고 대신 0을 반환하고 싶다.

In [1]:
def safe_divison(number, divisor,
                ignore_overflow,
                ignore_zero_division):
    
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [7]:
result = safe_divison(1.0, 10**500, True, False)
print(result)

0


In [8]:
result = safe_divison(1.0, 0, False, True)
print(result)

inf


문제는 어떤 예외를 무시할지 결정하는 두 불 변수의 위치를 혼동하기 쉽다는 것이다.

이로 인해 추적하기 힘든 버그가 생길 수 있다. 이 코드의 가독성을 향상시키는 방법은 키워드 인자를 사용하는 것이다.

In [9]:
def safe_divison(number, divisor,
                ignore_overflow=False,
                ignore_zero_division=False):
    
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [10]:
result = safe_divison(1.0, 10**500, ignore_overflow=True)
print(result)

0


In [11]:
result = safe_divison(1.0, 0, ignore_zero_division=True)
print(result)

inf


문제는 이런 식으로 키워드 인자를 사용하는 것이 선택적인 사항이므로 호출하는 쪽에서 명확성을 위해 키워드 인자를 꼭 쓰도록 강요할 수 없다는 데 있다.

이와 같이 복잡한 함수의 경우 호출자가 키워드만 사용하는 인자를 통해 의도를 명확히 밝히도록 요구하는 편이 좋다. 키워드만 사용하는 인자는 키워드를 반드시 사용해 지정해야 하며, 절대 위치를 기반으로는 지정할 수 없다.

다음은 safe_division 함수가 키워드만 사용하는 인자만 받도록 만든 코드다. 인자 목록에 있는 `*` 기호는 위치 인자의 마지막과 키워드만 사용하는 인자의 시작을 구분해준다.

In [12]:
def safe_divison_c(number, divisor, *,
                ignore_overflow=False,
                ignore_zero_division=False):
    
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [14]:
# 에러가 난다.

result = safe_divison_c(1.0, 10**500, True, False)
print(result)

TypeError: safe_divison_c() takes 2 positional arguments but 4 were given

In [16]:
# 이거 에러가 안 난다.

result = safe_divison(1.0, 0, ignore_zero_division=True)
print(result)

inf


하지만 이 safe_division_c 함수에도 문제가 있다.

호출하는 쪽에서 이 함수의 맨 앞에 있는 두 필수 인자(number, divisor)를 호출하면서 위치와 키워드를 혼용할 수 있다.

In [17]:
assert safe_divison_c(number=2, divisor=5) == 0.4
assert safe_divison_c(divisor=5, number=2) == 0.4
assert safe_divison_c(2, divisor=5) == 0.4

나중에 요구 사항이 바뀌거나 원하는 스타일이 바뀌어서 맨 앞의 두 인자 이름을 변경할 수도 있다. 그러면 호출 코드가 깨지는 이슈가 발생한다.

파이썬 3.8에는 이 문제에 대한 해법이 들어 있다. 이를 `위치로만 지정하는 인자` 라고 부른다.

위치로만 지정하는 인자는 반드시 위치만 사용해 인자를 지정 해야 하고 키워드 인자로는 쓸 수 없다.

다음 safe_division 함수는 처음 두 필수 인자를 위치로만 지정하는 인자로 지정한다. 인자 목록의 `/`  기호는 위치로만 지정하는 인자의 끝을 표시한다.

In [18]:
def safe_divison_d(number, divisor, /, *,
                ignore_overflow=False,
                ignore_zero_division=False):
    
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [19]:
assert safe_divison_d(2, 5) == 0.4

In [20]:
# 이거는 에러 발생한다.

safe_divison_d(number=2, divisor=5)

TypeError: safe_divison_d() got some positional-only arguments passed as keyword arguments: 'number, divisor'

이제 safe_division_d 함수 정의에서 맨 앞의 두 필수 인자는 호출하는 쪽과 분리 됐다. 따라서 파라미터 이름을 다시 바꿔도 아무것도 망가지지 않는다.

키워드로만 지정하는 인자와 위치로만 지정하는 인자로 인해 생기는 효과 중에서 언급할 만한 가지는 다음과 같다.

인자 목록에서 `/` 와 `*` 기호 사이에 있는 모든 파라미터는 위치를 사용해 전달할 수도 있고 이름을 키워드로 사용해 전달할 수도 있다.

예를 들어 다음 코드에서는 결과를 표시할 때 얼마나 많은 자릿수를 사용할지 결정하고자 반올림할 위치를 지정하는 인자를 safe_division에 추가한다.

In [21]:
def safe_divison_e(number, divisor, /, 
                   ndigits=10, *,
                   ignore_overflow=False,
                   ignore_zero_division=False):
    
    try:
        fraction = number / divisor
        return round(fraction, ndigits)
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

이 함수에서 ndigits 는 위치나 키워드를 사용해 전달할 수 있는 선택적인 파라미터이므로 어떤 방식으로 함수 호출에 사용할 수 있다.

In [22]:
result = safe_divison_e(22, 7)
print(result)

3.1428571429


In [23]:
result = safe_divison_e(22, 7, 5)
print(result)

3.14286


In [25]:
result = safe_divison_e(22, 7, ndigits=2)
print(result)

3.14


키워드로만 지정해야 하는 인자를 사용하면 호출하는 쪽에서 특정 인자를(위치를 사용 하지 않고) 반드시 키워드를 사용해 호출하도록 강제할 수 있다.
이로 인해 함수 호출의 의도를 명확히 할 수 있다. 키워드로만 지정해야 하는 인자는 인자 목록에서 `*` 다음에 위치한다.

위치로만 지정해야 하는 인자를 사용하면 호출하는 쪽에서 키워드를 사용해 인자를 지정하지 못하게 만들 수 있고, 이에 따라 함수 구현과 함수 호출 지점 사이의 결합을 줄일 수 있다.
위치로만 지정해야 하는 인자는 인자 목록에서 `/` 앞에 위치한다.

인자 목록에서 `/` 와 `*` 사이에 있는 파리미터는 키워드를 사용해 전달해도 되고 위치를 기반으로 전달해도 된다. 이런 동작은 파이썬 함수 파라미터의 기본 동작이다.